# Lab5: Morphosyntactic tagging

In [10]:
import collections
import locale
import math
import os
import requests
import spacy
import spacy.lang.pl
import spacy.tokenizer
import time

In [2]:
ACT_DIRECTORY = '../files'

## Download docker image of KRNNT2. It includes the following tools:
* Morfeusz2 - morphological dictionary
* Corpus2 - corpus access library
* Toki - tokenizer for Polish
* Maca - morphosyntactic analyzer
* KRNNT - Polish tagger

```
docker pull djstrong/krnnt2
docker run -it -p "9200:9200" djstrong/krnnt2 python3 /home/krnnt/krnnt/krnnt_serve.py /home/krnnt/krnnt/data
```

In [125]:
print(requests.post('http://localhost:9200', data='Ala ma kota.').text, end='')

Ala	none
	Ala	subst:sg:nom:f	disamb
ma	space
	mieć	fin:sg:ter:imperf	disamb
kota	space
	kot	subst:sg:acc:m2	disamb
.	none
	.	interp	disamb



## Use the tool to tag and lemmatize the law corpus.

Przy lematyzacji pominięte zostały wyrażenia zawierające więcej niż jedno słowo. W celu zachowania kontekstu przy dalszej analizie tymczasowo zastąpione zostały tokenem `X` klasy `X` (bigramy zawierające taki token zostaną pominięte przy dalszej analizie).

In [97]:
lemmatized_acts = {}

for root, _, files in os.walk(ACT_DIRECTORY):
    for file_name in files:
        path = os.path.join(root, file_name)
        with open(path, encoding='utf-8') as file:
            content = file.read()
            response = requests.post('http://localhost:9200', data=content.encode('utf-8')).text
            response = [line for line in response.split('\n') if line != '']
            lemmatized_content = []
            for lemmatized_word in zip(response[0::2], response[1::2]):
                lemmatized_word = ' '.join(lemmatized_word)
                split_lemmatized_word = lemmatized_word.split()
                if len(split_lemmatized_word) != 5:
                    print(f'Skipping "{lemmatized_word}"')
                    lemmatized_content += ['X:X']
                    continue
                _, _, token, category, _ = lemmatized_word.split()
                token = token.lower()
                category = category.split(':')[0]
                lemmatized_content += [f'{token}:{category}']
                
            lemmatized_acts[path] = lemmatized_content

Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	

Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "m3	none 	metr sześcienny	brev:npun	disamb"
Skipping "m3	none 	metr sześcienny	brev:npun	disamb"
Skipping "m3	space 	metr sześcienny	brev:npun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	none 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:p

Skipping "m3	none 	metr sześcienny	brev:npun	disamb"
Skipping "m3	none 	metr sześcienny	brev:npun	disamb"
Skipping "m3	space 	metr sześcienny	brev:npun	disamb"
Skipping "m3	space 	metr sześcienny	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m3	space 	metr sześcienny	brev:npun	disamb"
Skipping "ds	space 	do spraw	brev:pun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m3	space 	metr sześcienny	brev:npun	disamb"
Skipping "KM	space 	koń mechaniczny	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	br

Skipping "tj	space 	to jest	brev:pun	disamb"
Skipping "m2	none 	metr kwadratowy	brev:npun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "KM	space 	koń mechaniczny	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "tj	none 	to jest	brev:pun	disamb"
Skipping "J	none 	Ewangelia wg św. Jana	subst:sg:nom:f	disamb"
Skipping "Dz	none 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "Dz	space 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "Dz	space 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "Dz	space 	Dzieje Apostolskie	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "m2	space 	metr kwadratowy	brev:npun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "np	none 	na przykład	brev:pun	disamb"
Skipping "itp	space 	i tym podobne	brev:pun	disamb"
Skipping "Dz	none 	Dzieje Apostolskie	brev:npu

In [128]:
print(list(lemmatized_acts.values())[0][:50])

['dziennik:brev', '.:interp', 'ustawa:brev', '.:interp', 'z:prep', '2001:adj', 'rok:brev', '.:interp', 'numer:brev', '81:num', ',:interp', 'pozycja:brev', '.:interp', '873:adj', 'ustawa:subst', 'z:prep', 'dzień:subst', '8:adj', 'czerwiec:subst', '2001:adj', 'rok:brev', '.:interp', 'o:prep', 'zmiana:subst', 'ustawa:subst', 'o:prep', 'utworzyć:ger', 'agencja:subst', 'technika:subst', 'i:conj', 'technologia:subst', 'artykuł:brev', '.:interp', '1:adj', '.:interp', 'w:prep', 'ustawa:subst', 'z:prep', 'dzień:subst', '12:adj', 'kwiecień:subst', '1996:adj', 'rok:brev', '.:interp', 'o:prep', 'utworzyć:ger', 'agencja:subst', 'technika:subst', 'i:conj', 'technologia:subst']


## Using the tagged corpus compute bigram statistic for the tokens containing:
* lemmatized, downcased word
* morphosyntactic category of the word (subst, fin, adj, etc.)

Do zliczania bigramów wykorzystany został kod z poprzednich zajęć.

In [136]:
def count_ngrams(tokens, n=2):
    ngram_dict = collections.defaultdict(int)
    for words in zip(*[tokens[offset:] for offset in (range(n))]):
        ngram_dict[' '.join(words)] += 1
    return ngram_dict

In [137]:
bigram_counts = collections.defaultdict(int)

for _, lemmatized_act in lemmatized_acts.items():
    for bigram, count in count_ngrams(lemmatized_act).items():
        bigram_counts[bigram] += count

In [138]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]

[('artykuł:brev .:interp', 84174),
 ('ustęp:brev .:interp', 53363),
 ('pozycja:brev .:interp', 45454),
 (',:interp pozycja:brev', 43375),
 ('.:interp 1:adj', 40076),
 ('-:interp -:interp', 36549),
 ('rok:brev .:interp', 33171),
 ('w:prep artykuł:brev', 32182),
 (',:interp o:prep', 30027),
 ('o:prep który:adj', 28762),
 ('który:adj mowa:subst', 28644),
 ('mowa:subst w:prep', 28579),
 ('.:interp 2:adj', 26503),
 ('w:prep ustęp:brev', 23564),
 ('.:interp artykuł:brev', 23026),
 (',:interp w:prep', 22618),
 ('.:interp numer:brev', 21491),
 ('1:adj .:interp', 21391),
 ('2:adj .:interp', 21211),
 (',:interp z:prep', 20105)]

## Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.
 For example: "Ala ma kota", which is tagged as:
```
    Ala	none
            Ala	subst:sg:nom:f	disamb
    ma	space
            mieć	fin:sg:ter:imperf	disamb
    kota	space
            kot	subst:sg:acc:m2	disamb
    .	none
            .	interp	disamb
```
the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

Pominięte zostały wyrazy zawierające znaki inne niż alfanumeryczne oraz rekordy formatu `X:X` wspomniane we wcześniejszym punkcie.

In [143]:
bigram_counts = {
    bigram: count 
    for (bigram, count) 
    in bigram_counts.items()
    if all([token.split(':')[0].isalpha() and token != 'X:X' for token in bigram.split(' ')])
}

In [144]:
sorted(bigram_counts.items(), key=lambda bigram_count: -bigram_count[1])[:20]

[('w:prep artykuł:brev', 32182),
 ('o:prep który:adj', 28762),
 ('który:adj mowa:subst', 28644),
 ('mowa:subst w:prep', 28579),
 ('w:prep ustęp:brev', 23564),
 ('z:prep dzień:subst', 11416),
 ('otrzymywać:fin brzmienie:subst', 10586),
 ('określić:ppas w:prep', 9760),
 ('do:prep sprawa:subst', 8734),
 ('ustawa:subst z:prep', 8676),
 ('właściwy:adj do:prep', 8555),
 ('i:conj numer:brev', 8464),
 ('dodawać:fin się:qub', 8236),
 ('minister:subst właściwy:adj', 7949),
 ('w:prep brzmienie:subst', 7326),
 ('w:prep droga:subst', 7138),
 ('w:prep przypadek:subst', 6840),
 ('na:prep podstawa:subst', 6727),
 ('stosować:fin się:qub', 6565),
 ('się:qub wyraz:subst', 6077)]

## Compute LLR statistic for this dataset.

In [104]:
def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])

In [105]:
bigram_first_token_count = collections.defaultdict(int)
bigram_second_token_count = collections.defaultdict(int)

for bigram, count in bigram_counts.items():
    first_token, second_token = bigram.split(' ')
    bigram_first_token_count[first_token] += count
    bigram_second_token_count[second_token] += count

In [107]:
total_bigram_count = sum(bigram_counts.values())    

bigram_llr = {}

for bigram, count in bigram_counts.items():
    first_token, second_token = bigram.split(' ')
    k11 = count
    k12 = bigram_second_token_count[second_token] - count
    k21 = bigram_first_token_count[first_token] - count
    k22 = total_bigram_count - (k11 + k12 + k21)
    
    bigram_llr[bigram] = llr_2x2(k11, k12, k21, k22)

In [122]:
sorted(bigram_llr.items(), key=lambda bigram_llr: -bigram_llr[1])[:10]

[('który:adj mowa:subst', 249005.74580686435),
 ('o:prep który:adj', 191025.1762959935),
 ('mowa:subst w:prep', 177761.7772774303),
 ('w:prep artykuł:brev', 114320.77819801657),
 ('otrzymywać:fin brzmienie:subst', 111262.17472637648),
 ('w:prep ustęp:brev', 88422.20964241377),
 ('minister:subst właściwy:adj', 71030.62874013692),
 ('dodawać:fin się:qub', 66959.43016569444),
 ('i:conj numer:brev', 54943.279354307335),
 ('droga:subst rozporządzenie:subst', 53974.92283251611)]

## Partition the entries based on the syntactic categories of the words, i.e. all bigrams having the form of w1:adj w2:subst should be placed in one partition (the order of the words may not be changed).

In [113]:
partitions = collections.defaultdict(dict)

for bigram, count in bigram_counts.items():
    entry1, entry2 = bigram.split(' ')
    w1, cat1 = entry1.split(':')
    w2, cat2 = entry2.split(':')
    partitions[(cat1, cat2)][bigram] = count

In [148]:
list(partitions.keys())[:10]

[('subst', 'prep'),
 ('prep', 'subst'),
 ('subst', 'subst'),
 ('prep', 'ger'),
 ('ger', 'subst'),
 ('subst', 'conj'),
 ('conj', 'subst'),
 ('subst', 'brev'),
 ('conj', 'prep'),
 ('fin', 'qub')]

In [119]:
list(list(partitions.values())[0].items())[:10]

[('ustawa:subst z:prep', 8676),
 ('ustawa:subst o:prep', 1696),
 ('przypadek:subst w:prep', 23),
 ('mowa:subst w:prep', 28579),
 ('przedstawiciel:subst do:prep', 17),
 ('uzgodnienie:subst z:prep', 181),
 ('miesiąc:subst od:prep', 1590),
 ('życie:subst po:prep', 822),
 ('dzień:subst od:prep', 2517),
 ('senat:subst bez:prep', 117)]

##  Select the 10 largest partitions (partitions with the larges number of entries).

Osobno wzięto pod uwagę rozmiar klasy pod względem sumy wystąpień wszystkich bigramów oraz pod względem liczby różnych bigramów.

### Suma wystąpień

In [161]:
partition_sums = {}

for partition, bigram_counts in partitions.items():
    partition_sum = 0
    for _, count in bigram_counts.items():
        partition_sum += count
    
    partition_sums[partition] = partition_sum

In [162]:
sorted(partition_sums.items(), key=lambda partition_sum: -partition_sum[1])[:10]

[(('prep', 'subst'), 327849),
 (('subst', 'subst'), 293799),
 (('subst', 'adj'), 274968),
 (('adj', 'subst'), 188440),
 (('subst', 'prep'), 173782),
 (('subst', 'conj'), 85165),
 (('conj', 'subst'), 84267),
 (('prep', 'adj'), 79486),
 (('ger', 'subst'), 76537),
 (('prep', 'brev'), 67135)]

### Liczba różnych bigramów

In [159]:
partition_sizes = {}

for partition, bigram_counts in partitions.items():
    partition_sizes[partition] = len(bigram_counts.items())  

In [163]:
sorted(partition_sizes.items(), key=lambda partition_sum: -partition_sum[1])[:10]

[(('subst', 'subst'), 48919),
 (('subst', 'adj'), 27300),
 (('adj', 'subst'), 26249),
 (('subst', 'fin'), 16303),
 (('ger', 'subst'), 15458),
 (('prep', 'subst'), 12324),
 (('subst', 'prep'), 11404),
 (('subst', 'ppas'), 10759),
 (('fin', 'subst'), 8826),
 (('adj', 'fin'), 8704)]

##   Use the computed LLR measure to select 5 bigrams for each of the largest categories.

Podobnie jak w poprzednim podpunkcie osobno wzięto pod uwagę rozmiar klasy pod względem sumy wystąpień wszystkich bigramów oraz pod względem liczby wystąpień różnych bigramów.

In [153]:
partition_bigram_llr = collections.defaultdict(dict)

for bigram, llr in bigram_llr.items():
    entry1, entry2 = bigram.split(' ')
    w1, cat1 = entry1.split(':')
    w2, cat2 = entry2.split(':')
    
    partition_bigram_llr[(cat1, cat2)][bigram] = llr

### Suma wystąpień

In [164]:
for partition, _ in sorted(partition_sums.items(), key=lambda partition_sum: -partition_sum[1])[:10]:
    print(f'* {" + ".join(partition)}')
    for bigram, llr in sorted(partition_bigram_llr[partition].items(), key=lambda bigram_llr: -bigram_llr[1])[:5]:
        print(f'\t* {bigram}, {llr}')

* prep + subst
	* z:prep dzień:subst, 53719.905124070356
	* na:prep podstawa:subst, 47389.81442574458
	* do:prep sprawa:subst, 46331.31717067293
	* w:prep droga:subst, 32059.638214459876
	* od:prep dzień:subst, 31767.93208489736
* subst + subst
	* droga:subst rozporządzenie:subst, 53974.92283251611
	* skarb:subst państwo:subst, 21933.487731331115
	* rada:subst minister:subst, 18342.685474431666
	* terytorium:subst rzeczpospolita:subst, 14155.022492691452
	* ochrona:subst środowisko:subst, 14029.946469116025
* subst + adj
	* minister:subst właściwy:adj, 71030.62874013692
	* rzeczpospolita:subst polski:adj, 41741.47174964618
	* jednostka:subst organizacyjny:adj, 24609.37429213093
	* samorząd:subst terytorialny:adj, 23394.129359848223
	* produkt:subst leczniczy:adj, 21913.360666442954
* adj + subst
	* który:adj mowa:subst, 249005.74580686435
	* niniejszy:adj ustawa:subst, 21509.33810154465
	* następujący:adj zmiana:subst, 18176.769159664094
	* odrębny:adj przepis:subst, 13058.805740462412

### Liczba różnych bigramów

In [165]:
for partition, _ in sorted(partition_sizes.items(), key=lambda partition_size: -partition_size[1])[:10]:
    print(f'* {" + ".join(partition)}')
    for bigram, llr in sorted(partition_bigram_llr[partition].items(), key=lambda bigram_llr: -bigram_llr[1])[:5]:
        print(f'\t* {bigram}, {llr}')

* subst + subst
	* droga:subst rozporządzenie:subst, 53974.92283251611
	* skarb:subst państwo:subst, 21933.487731331115
	* rada:subst minister:subst, 18342.685474431666
	* terytorium:subst rzeczpospolita:subst, 14155.022492691452
	* ochrona:subst środowisko:subst, 14029.946469116025
* subst + adj
	* minister:subst właściwy:adj, 71030.62874013692
	* rzeczpospolita:subst polski:adj, 41741.47174964618
	* jednostka:subst organizacyjny:adj, 24609.37429213093
	* samorząd:subst terytorialny:adj, 23394.129359848223
	* produkt:subst leczniczy:adj, 21913.360666442954
* adj + subst
	* który:adj mowa:subst, 249005.74580686435
	* niniejszy:adj ustawa:subst, 21509.33810154465
	* następujący:adj zmiana:subst, 18176.769159664094
	* odrębny:adj przepis:subst, 13058.805740462412
	* walny:adj zgromadzenie:subst, 9635.448067530888
* subst + fin
	* kropka:subst zastępować:fin, 10276.572657771114
	* ustawa:subst wchodzić:fin, 9366.649872020353
	* treść:subst oznaczać:fin, 2861.1177047609344
	* minister:subs

##  Using the results from the previous step answer the following questions:
### What types of bigrams have been found?
Prawie wszystkie z największych kategorii zawierają na którejś pozycji rzeczownik. Jeśli za "dużą kategorię" uznamy taką kategorię, w której suma wystąpień wszystkich bigramów jest największa, to najpopularniejsza okazała się kategoria "przyimek + rzeczownik", jednak jeśli spojrzymy na kategorie pod kątem różnorodności i mnogości wystąpień różnych bigramów, to najwięcej wystąpień ma para "rzeczownik + rzeczownik".

### Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?
Najbardziej wartościowe wydają się bigramy zawierające rzeczownik. Choć wyrażenia typu "przyimek + rzeczownik" występują w języku polskim bardzo często, to wydaje mi się, że ciekawsze semantycznie są nazwy własne oparte o bigramy "rzeczownik + rzeczownik" (jak "skarb państwa") czy "przymiotnik + rzeczownik" (jak "produkt leczniczy").

### Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
Jak można było zauważyć na poprzednich zajęciach, sam LLR trochę "tonie" w spójnikach i przyimkach. W połączeniu z kategoriami bigramów można dużo łatwiej wyłonić najbardziej wartościowe i popularne wyrażenia, stosując LLR właśnie w obrębie tych kategorii.

### Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?
* wyszukiwanie keywordów,
* analiza nazw własnych,
* tłumaczenia - wystąpienie słowa w konkretnym wyrażniu ma duże znaczenie przy tłumaczeniach.
